<a href="https://colab.research.google.com/github/PhatthanhtuHN/ML_Echocardiographic/blob/main/ML_echocardiographic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
from torch import nn

from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import numpy as np

from torchsummary import summary

from collections import  namedtuple

import matplotlib.pyplot as plt

from google.colab import drive

from sklearn.metrics import classification_report

In [9]:
drive.mount('/content/drive')

# Đặt đường dẫn cho thư mục train và test từ dữ liệu Drive đã được kết nối 
train_dir = '/content/drive/MyDrive/Colab Notebooks/DATA_CHAMBER_2021/train'
test_dir = '/content/drive/MyDrive/Colab Notebooks/DATA_CHAMBER_2021/test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Hàm get_classes() trả về ba lớp 2C, 3C, 4C tương ứng với ba loại mặt cắt của ảnh siêu âm tim
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():
  img_size = 224
  # Hàm transform chuẩn hóa ảnh về kích thước 224 x 224 và chuyển thành Tensor là một mảng nhiều chiều
  transforms_train = transforms.Compose([
    transforms.Resize((img_size, img_size)), 
    transforms.ToTensor(), 
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                  std=[0.229, 0.224, 0.225]) 
  ])
  transforms_test = transforms.Compose([
    transforms.Resize((img_size, img_size)), 
    transforms.ToTensor(), 
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                  std=[0.229, 0.224, 0.225]) 
  ])
  # Chuẩn bị 2 bộ dữ liệu trainset và testset với đường dẫn như trên, gán nhãn cho ảnh theo tên thư mục và trả về bộ giá trị (tuple) TrainTest gồm hai bộ dữ liệu này
  trainset = torchvision.datasets.ImageFolder(root=train_dir, transform=transforms_train)
  testset = torchvision.datasets.ImageFolder(root=test_dir, transform=transforms_test)
  print('Number of images in train:', len(trainset), '\nNumber of images in test:', len(testset))
  print('Index of classes:', trainset.class_to_idx)
  print('Type:', type(trainset[0][0]), '| size: ', trainset[0][0].shape)
  return TrainTest(train=trainset, test=testset)

# Hàm đọc bộ dữ liệu, mỗi lần lấy ra 1 batch có độ dài là 32 ảnh đế xử lý song song, nối thành tensor 4 chiều là đầu vào của mạng
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  print('Number of batchs in train:', len(trainloader), '\nNumber of batchs in test:', len(testloader))
  return TrainTest(train=trainloader, test=testloader)

# Dựng mô hình mạng tích chập VGG19
class VGG19(nn.Module):
  def __init__(self):
    super().__init__()
    self.features = self._make_features()
    self.classification_head = nn.Linear(in_features=25088, out_features=3)

  def forward(self, x):
    out = self.features(x)
    out = out.view(out.size(0), -1)
    out = self.classification_head(out)
    return out

  def _make_features(self):
    config = [64, 64, 'MP', 128, 128, 'MP', 256, 256, 256, 256, 'MP', 512, 512, 512, 512, 'MP', 512, 512, 512, 512, 'MP']
    layer = []
    c_in = 3
    for c in config:
      if c == 'MP':
        layer += [nn.MaxPool2d(kernel_size=2, stride=2)]
      else:
        layer += [nn.Conv2d(in_channels=c_in, out_channels=c, kernel_size=3, stride=1, padding=1), 
                  nn.BatchNorm2d(num_features=c), 
                  nn.ReLU6(inplace=True)]
        c_in = c
    return nn.Sequential(*layer)


In [11]:
def imshow(images, labels, predicted, target_names):
  img = torchvision.utils.make_grid(images)
  plt.imshow(img.permute(1, 2, 0).cpu().numpy())
  [print(target_names[c], end=' ') for c in list(labels.cpu().numpy())]
  print()
  [print(target_names[c], end=' ') for c in list(predicted.cpu().numpy())]
  print()
  # print(target_names[list(labels.cpu().numpy())])
  # print(target_names[list(predicted.cpu().numpy())])

  # def imshow(img):
  #   img = img / 2 + 0.5     # unnormalize
  #   npimg = img.numpy()
  #   plt.imshow(np.transpose(npimg, (1, 2, 0)))
  #   plt.show()

In [12]:
# Hàm train với mỗi batch trong bộ dữ liệu
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  # Cho ảnh (đã được chia thành batch trong loader) lần lượt đi qua model, với mỗi ảnh và nhãn trong bộ dữ liệu là đầu vào của mạng.
  model.train()             
  running_loss = 0.0
  reporting_steps = 60
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    # print('Type:', type(outputs), '\nOutput shape:', outputs.size())      # Type: <class 'torch.Tensor'> Output shape: torch.Size([32, 3])
    loss = loss_func(outputs, labels)

    # Cập nhật tham số trước khi sang step khác
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    # Báo cáo sau reporting_step bước
    if i % reporting_steps == reporting_steps-1:
      print(f'Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}')
      running_loss = 0.0

# Hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue =[]
  ypred = []
  # Đặt model ở chế độ đánh giá (evaluate)
  with torch.no_grad():
    model.eval()
    
    # Tính toán đầu ra cho từng ảnh, với nhãn dự đoán dựa trên số to nhất trong outputs
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())
      
  return ytrue, ypred   # Trả về nhãn thực, nhãn dự đoán của ảnh


In [17]:
def main():
  classes = get_classes()
  datasets = prepare_data()

  # img, label = datasets.train[0]
  # print(classes[label], img.size)
  # print('Number of images in train', len(datasets.train), 'Number of images in test', len(datasets.test))
  # plt.imshow(img)
  loaders = prepare_loader(datasets)
  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  model = VGG19().to(device)
  # summary(model.cuda(), (3, 224, 224))
  # images, labels = iter(loaders.train).next()
  # # print(model)
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)

  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ytrue, ypred = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

main()


Number of images in train: 6717 
Number of images in test: 1607
Index of classes: {'2C': 0, '3C': 1, '4C': 2}
Type: <class 'torch.Tensor'> | size:  torch.Size([3, 224, 224])
Number of batchs in train: 210 
Number of batchs in test: 51


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 step 59 ave_loss 9.2767
Epoch 0 step 119 ave_loss 2.5113
Epoch 0 step 179 ave_loss 2.1114
              precision    recall  f1-score   support

          2C       0.54      0.32      0.40       409
          3C       0.54      0.57      0.55       367
          4C       0.75      0.87      0.80       831

    accuracy                           0.66      1607
   macro avg       0.61      0.59      0.59      1607
weighted avg       0.64      0.66      0.64      1607

Epoch 1 step 59 ave_loss 1.2977
Epoch 1 step 119 ave_loss 0.9737
Epoch 1 step 179 ave_loss 0.6869
              precision    recall  f1-score   support

          2C       0.40      0.75      0.52       409
          3C       0.54      0.59      0.56       367
          4C       0.83      0.45      0.58       831

    accuracy                           0.56      1607
   macro avg       0.59      0.59      0.56      1607
weighted avg       0.66      0.56      0.56      1607

Epoch 2 step 59 ave_loss 0.6116
Epoch 2 st

In [ ]:
#VGG19  ảnh 224: [56, 67, 62, 65, 63, 73, 77, 77, 72]